# DataSens E1_v1 — 02_schema_create

- Objectifs: créer le schéma (SQLite), 10 tables Pareto
- Prérequis: exécuter 01_setup_env
- Sortie: base `datasens/datasens.db`
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - SQLite (fichier local) simplifie la démo E1_v1.
> - `sqlite3.connect(DB_PATH)` ouvre (ou crée) la base.
> - `CREATE TABLE IF NOT EXISTS` évite les erreurs si la table existe déjà.
> - `conn.commit()` enregistre les changements.
> - Une ligne est ajoutée dans `README_VERSIONNING.md` pour la traçabilité.
> - Les tables couvrent les entités principales collecte/gouvernance.


In [ ]:
# DataSens E1_v1 - 02_schema_create
# 💾 Base SQLite et schéma (18 tables)
import sqlite3
from datetime import datetime
from pathlib import Path

# Détection du dossier projet (compat exécution depuis notebooks/)
PROJECT_ROOT = Path.cwd().parents[2] if (Path.cwd().name.startswith('datasens_')) else Path.cwd()
# Emplacement du fichier SQLite (BDD locale de démo E1_v1)
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

# Connexion à la base SQLite (créée si absente)
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
print(f"✅ Base SQLite prête : {DB_PATH}")

# Tables principales (collecte/gouvernance) inspirées du monolithe
TABLES_SQL = {
    # Typologie des données (RSS, API, Scraping, CSV, Big Data)
    "type_donnee": """
    CREATE TABLE IF NOT EXISTS type_donnee (
        id_type_donnee INTEGER PRIMARY KEY AUTOINCREMENT,
        libelle TEXT, description TEXT,
        frequence_maj TEXT, categorie_metier TEXT
    );""",
    # Source (provenance) rattachée à un type de donnée
    "source": """
    CREATE TABLE IF NOT EXISTS source (
        id_source INTEGER PRIMARY KEY AUTOINCREMENT,
        id_type_donnee INTEGER,
        nom TEXT, url TEXT, fiabilite REAL,
        FOREIGN KEY (id_type_donnee) REFERENCES type_donnee(id_type_donnee)
    );""",
    # Flux (opération de collecte datée + format + manifest)
    "flux": """
    CREATE TABLE IF NOT EXISTS flux (
        id_flux INTEGER PRIMARY KEY AUTOINCREMENT,
        id_source INTEGER, date_collecte TEXT, format TEXT, manifest_uri TEXT,
        FOREIGN KEY (id_source) REFERENCES source(id_source)
    );""",
    # Document (contenu final) rattaché à un flux
    "document": """
    CREATE TABLE IF NOT EXISTS document (
        id_doc INTEGER PRIMARY KEY AUTOINCREMENT,
        id_flux INTEGER, titre TEXT, texte TEXT, langue TEXT, date_publication TEXT,
        FOREIGN KEY (id_flux) REFERENCES flux(id_flux)
    );""",
    # Référentiels territoriaux et indicateurs (simplifiés E1)
    "pays": "CREATE TABLE IF NOT EXISTS pays (id_pays INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT);",
    "region": "CREATE TABLE IF NOT EXISTS region (id_region INTEGER PRIMARY KEY AUTOINCREMENT, id_pays INTEGER, nom TEXT, FOREIGN KEY (id_pays) REFERENCES pays(id_pays));",
    "departement": "CREATE TABLE IF NOT EXISTS departement (id_departement INTEGER PRIMARY KEY AUTOINCREMENT, id_region INTEGER, code_dept TEXT, nom TEXT, FOREIGN KEY (id_region) REFERENCES region(id_region));",
    "commune": "CREATE TABLE IF NOT EXISTS commune (id_commune INTEGER PRIMARY KEY AUTOINCREMENT, id_departement INTEGER, code_insee TEXT, nom_commune TEXT, lat REAL, lon REAL, population INTEGER, FOREIGN KEY (id_departement) REFERENCES departement(id_departement));",
    "territoire": "CREATE TABLE IF NOT EXISTS territoire (id_territoire INTEGER PRIMARY KEY AUTOINCREMENT, id_commune INTEGER, FOREIGN KEY (id_commune) REFERENCES commune(id_commune));",
    "type_indicateur": "CREATE TABLE IF NOT EXISTS type_indicateur (id_type_indic INTEGER PRIMARY KEY AUTOINCREMENT, code TEXT, libelle TEXT, unite TEXT);",
    "indicateur": "CREATE TABLE IF NOT EXISTS indicateur (id_indic INTEGER PRIMARY KEY AUTOINCREMENT, id_territoire INTEGER, id_type_indic INTEGER, valeur REAL, annee INTEGER, FOREIGN KEY (id_territoire) REFERENCES territoire(id_territoire), FOREIGN KEY (id_type_indic) REFERENCES type_indicateur(id_type_indic));",
    "meteo": "CREATE TABLE IF NOT EXISTS meteo (id_meteo INTEGER PRIMARY KEY AUTOINCREMENT, id_territoire INTEGER, date_obs TEXT, temperature REAL, humidite REAL, vent_kmh REAL, pression REAL, FOREIGN KEY (id_territoire) REFERENCES territoire(id_territoire));",
    "utilisateur": "CREATE TABLE IF NOT EXISTS utilisateur (id_user INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, role TEXT, organisation TEXT);",
    "emotion": "CREATE TABLE IF NOT EXISTS emotion (id_emotion INTEGER PRIMARY KEY AUTOINCREMENT, type_emotion TEXT, valence TEXT, score_confiance REAL);",
    "annotation": "CREATE TABLE IF NOT EXISTS annotation (id_annotation INTEGER PRIMARY KEY AUTOINCREMENT, id_doc INTEGER, id_user INTEGER, id_emotion INTEGER, intensity REAL, polarity TEXT, date_annotation TEXT, FOREIGN KEY (id_doc) REFERENCES document(id_doc), FOREIGN KEY (id_user) REFERENCES utilisateur(id_user), FOREIGN KEY (id_emotion) REFERENCES emotion(id_emotion));",
    "theme": "CREATE TABLE IF NOT EXISTS theme (id_theme INTEGER PRIMARY KEY AUTOINCREMENT, libelle TEXT, description TEXT);",
    "evenement": "CREATE TABLE IF NOT EXISTS evenement (id_event INTEGER PRIMARY KEY AUTOINCREMENT, id_theme INTEGER, date_event TEXT, avg_tone REAL, source_event TEXT, FOREIGN KEY (id_theme) REFERENCES theme(id_theme));",
    "document_evenement": "CREATE TABLE IF NOT EXISTS document_evenement (id_doc INTEGER, id_event INTEGER, PRIMARY KEY (id_doc, id_event), FOREIGN KEY (id_doc) REFERENCES document(id_doc), FOREIGN KEY (id_event) REFERENCES evenement(id_event));"
}

# Exécution séquentielle des DDL
for name, sql in TABLES_SQL.items():
    cur.execute(sql)
    print(f"✅ Table : {name}")
conn.commit()

# Journalisation (traçabilité pour le jury)
with open(PROJECT_ROOT / "README_VERSIONNING.md", "a", encoding="utf-8") as f:
    f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_SCHEMA` | Création des 18 tables principales\n")
print("Schéma créé.")
